# Example of using C++ code in python and Jupyter with SWIG

It is also possible to use our C++ code from python and Jupyter. This involves using the [SWIG](http://www.swig.org) package. You can download it [here](http://www.swig.org/survey.html) and then install following instructions [here](http://www.swig.org/Doc3.0/Preface.html#Preface_installation). If you are successful, you should be able to open a new terminal and type ```which swig``` to obtain the path of swig. 

The idea is then to use SWIG to automatically generate python-readable code from our C++/C libraries. There is a lot to learn in this regard, so we will try first with a simple example that illustrates some concepts we will need, such as using STL libraries and C++11 compilation. 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numba



## Step 1 : Look at C++ files

You should be able to see these two simple C++ files: 


In [ ]:
! cat swig_example/example.hpp swig_example/example.cpp 



## Step 2 : Look at SWIG interface file

The magic of SWIG is to create wrapper C++ functions that use the "cython" interface. You will see an "interface" file for SWIG :

In [ ]:
! cat swig_example/example.i 

## Step 3 : Look at SWIG setup file

This tells SWIG to generate a file called ```example_wrap.cc``` using source from ```example.cpp``` with C++11. 


In [ ]:
! cat swig_example/setup.py

Now create the interface using the interface file. This will create two files, ```example_wrap.cxx``` and ```example.py```. They are not intended to be human-readable, so we don't need to look into them. 

## Step 4: Create SWIG interface from file

In [ ]:
! swig -c++ -python swig_example/example.i 

## Step 5 : compile C++ and interface

This will compile our file ```example.cpp``` along with ```example_wrap.cxx``` using the rules we set (for instance, using the C++11 compiler). 

We are now free to use this in python!

In [ ]:
! python swig_example/setup.py build_ext --inplace


## Step 6: Make profit: 

First we import the path correctly. 

In [ ]:
import sys
import os

sys.path.append( os.path.abspath("swig_example") )

print (sys.path)




Next we import our actual function from "example.py"

In [ ]:
import example
help(example)

And finally, we can use our code: 

In [ ]:
a = example.vector_int([1,2,3])
x = example.sum_int(a)
print (x)


# Success!
Now let's compare with bare python, numba, and numpy.

In [ ]:
import numba
import numpy as np

def sum_int_py(arr):
    total = 0
    for x in arr:
        total += x
    return total

@numba.jit(nopython=True)
def sum_int_numba(arr):
    total = 0
    for x in arr:
        total += x
    return total
sum_int_py([0, 1, 2, 3]) # Run it once to force compilation

In [ ]:
arr_np = np.arange(1_000_000)
print("Bare python:")
%timeit sum_int_py(arr_np)

print("numpy built-in:")
%timeit np.sum(arr_np)

print("numba:")
%timeit sum_int_numba(arr_np)

arr_cpp = example.vector_int(range(1000000))
print("swig:")
%timeit example.sum_int(arr_cpp)

In [ ]:
# Finally, map out the performance to inspect the overhead
xx = np.array([10**i for i in range(8)])
t_py = np.zeros_like(xx, dtype=float)
dt_py = np.zeros_like(xx, dtype=float)
t_numba = np.zeros_like(xx, dtype=float)
dt_numba = np.zeros_like(xx, dtype=float)
t_numpy = np.zeros_like(xx, dtype=float)
dt_numpy = np.zeros_like(xx, dtype=float)
t_swig = np.zeros_like(xx, dtype=float)
dt_swig = np.zeros_like(xx, dtype=float)

for i in range(len(xx)):
    print(f"Test length {xx[i]}")
    print("Bare python")
    arr_np = np.arange(xx[i])
    result_py = %timeit -o sum_int_py(arr_np)
    t_py[i] = result_py.average
    dt_py[i] = result_py.stdev

    print("Numba")
    result_numba = %timeit -o sum_int_numba(arr_np)
    t_numba[i] = result_numba.average
    dt_numba[i] = result_numba.stdev

    print("Numpy")
    result_numpy = %timeit -o np.sum(arr_np)
    t_numpy[i] = result_numpy.average
    dt_numpy[i] = result_numpy.stdev

    print("Swig")
    arr_cpp = example.vector_int(range(xx[i]))
    result_swig = %timeit -o example.sum_int(arr_cpp)
    t_swig[i] = result_swig.average
    dt_swig[i] = result_swig.stdev


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.errorbar(xx, t_py, yerr=dt_py, color="red", label="Bare python")
ax.errorbar(xx, t_numba, yerr=dt_numba, color="blue", label="Numba")
ax.errorbar(xx, t_numpy, yerr=dt_numpy, color="green", label="Numpy")
ax.errorbar(xx, t_swig, yerr=dt_swig, color="purple", label="Swig")
ax.set_xlabel("Array length")
ax.set_xscale("log")
ax.set_ylabel("Time [s]")
ax.set_yscale("log")
ax.tick_params(which='both', direction="in", top=True, bottom=True, left=True, right=True)
ax.legend(loc="upper left")